## 3.1 Data Preparation

### 3.1.1 Import & Standardization of Raw CSV Files

In [ ]:
import pandas as pd
import numpy as np

# ------------------------------------------------------------
# Helper functions
# ------------------------------------------------------------

def load_csv(path):
    """
    Load a CSV file with common options and lowercase columns.
    """
    df = pd.read_csv(path, sep=",") # based on csv separator you use
    df.columns = df.columns.str.strip().str.lower()  # normalize column names
    return df


def enforce_string(df, cols):
    """
    Convert selected columns to string type.
    """
    for c in cols:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip()
    return df


def enforce_float(df, cols):
    """
    Convert selected columns to numeric (float), coercing invalid values to NaN.
    """
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df


def enforce_datetime(df, cols):
    """
    Convert selected columns to datetime.
    """
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors="coerce")
    return df

def divider(title):
    print(f"\n{'='*20} {title} {'='*20}\n")


In [27]:
# ------------------------------------------------------------
# File paths (modify if needed)
# ------------------------------------------------------------

path_collar    = "/home/achmad/Projects/data_analyst/EDA_for_NickelExploration/data/raw_collar.csv"
path_assay     = "/home/achmad/Projects/data_analyst/EDA_for_NickelExploration/data/raw_assay.csv"
path_geology    = "/home/achmad/Projects/data_analyst/EDA_for_NickelExploration/data/raw_geology.csv"

# ------------------------------------------------------------
# 1. Import CSV files
# ------------------------------------------------------------

collar_df    = load_csv(path_collar)
assay_df     = load_csv(path_assay)
geo_df     = load_csv(path_geology)

divider("Collar")
print(collar_df)

divider("Assay")
print(assay_df)

divider("Lithology")
print(geo_df)


==================== Collar ====================

      hole_id   date   easting_x  northing_y  elevation   eoh  inc_samp
0     E07-017  44545  404120.680  114875.100    375.571  16.0        18
1     E07-018  44394  404127.929  114849.522    383.153  30.0        34
2     E07-019  44545  404157.778  114865.303    371.663  16.0        16
3     E07-020  44394  404170.353  114853.791    371.346  10.5        15
4     E07-021  44545  404208.368  114873.263    376.459  19.5        21
..        ...    ...         ...         ...        ...   ...       ...
607  F09-037i  45453  404631.760  114017.846    532.605  14.0        16
608  F09-039i  45453  404681.747  114023.668    522.609  10.0        10
609  F09-040A  45453  404675.884  113993.707    530.254  15.0        16
610  F09-041i  45453  404728.662  114021.932    523.592  10.0        10
611  F09-043i  45453  404777.901  114026.667    507.087  10.0        10

[612 rows x 7 columns]

==================== Assay ====================

           

In [28]:
# # ------------------------------------------------------------
# # 2. Standardize column types
# # ------------------------------------------------------------

# ID fields that must be string (modify based on your schema)
string_columns = {
    "collar":    ["hole_id"],
    "assay":    [
        "sample_id", 
        "hole_id"
    ],
    "geology":    [
        "sample_id", 
        "hole_id",
        "length"
    ],    

}

# # Numeric fields
numeric_columns = {
    "collar":    [
        "easting_x", 
        "northing_y", 
        "elevation", 
        "eoh", 
        "inc_samp"
    ],
    "assay":    [
        "depth_from", 
        "depth_to", 
        "core_length", 
        "ni", 
        "fe",
        "mgo",
        "sio2"
    ],
    "geology":    [
        "depth_from", 
        "depth_to", 
        "core_length", 
    ],     
}

# # Date fields
date_columns = {
    "collar": ["date"],
}

# # ------------------------------------------------------------
# # Apply to each dataset
# # ------------------------------------------------------------

# collar
collar_df = enforce_string(collar_df, string_columns["collar"])
collar_df = enforce_float(collar_df, numeric_columns["collar"])
collar_df = enforce_datetime(collar_df, date_columns["collar"])

# assay
assay_df = enforce_string(assay_df, string_columns["assay"])
assay_df = enforce_float(assay_df, numeric_columns["assay"])

# geology
geo_df = enforce_string(geo_df, string_columns["geology"])
geo_df = enforce_float(geo_df, numeric_columns["geology"])

# # ------------------------------------------------------------
# # 3. Final output summary
# # ------------------------------------------------------------
divider("Collar")
collar_df.info()

divider("Assay")
assay_df.info()

divider("Lithology")
geo_df.info()


==================== Collar ====================

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612 entries, 0 to 611
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   hole_id     612 non-null    object        
 1   date        612 non-null    datetime64[ns]
 2   easting_x   612 non-null    float64       
 3   northing_y  612 non-null    float64       
 4   elevation   612 non-null    float64       
 5   eoh         612 non-null    float64       
 6   inc_samp    612 non-null    int64         
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 33.6+ KB

==================== Assay ====================

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9190 entries, 0 to 9189
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sample_id    9190 non-null   object 
 1   hole_id      9190 non-null   object 
 2   d

### 3.1.2 Data Cleaning & Quality Validation

In [29]:
import pandas as pd
import numpy as np

# ------------------------------------------------------------
# Helper functions
# ------------------------------------------------------------

def divider(title):
    print(f"\n{'='*20} {title} {'='*20}\n")

# 1. Duplicate removal
def remove_duplicates(df, subset):
    before = len(df)

    # Identify duplicates BEFORE removal
    duplicates = df[df.duplicated(subset=subset, keep=False)].copy()

    df_clean = df.drop_duplicates(subset=subset)
    removed = before - len(df_clean)

    print(f"Removed {removed} duplicates based on {subset}")

    return df_clean, duplicates


# 2. Normalize missing values
def normalize_missing(df, essential_cols=None):
    error_tokens = ["#N/A", "N/A", "NA", "", "-", "--", "null", "NULL"]
    df = df.replace(error_tokens, np.nan)

    if essential_cols:
        missing_rows = df[df[essential_cols].isna().any(axis=1)].copy()
        df_clean = df.dropna(subset=essential_cols)
    else:
        missing_rows = pd.DataFrame()
        df_clean = df

    print(f"Missing essential rows removed: {len(missing_rows)}")

    return df_clean, missing_rows


# 3. Validate intervals
def validate_intervals(df, name):
    if not {"depth_from", "depth_to"}.issubset(df.columns):
        print(f"{name}: missing depth interval columns.")
        return df, pd.DataFrame()

    df["invalid_interval"] = (df["depth_from"] >= df["depth_to"])

    invalid_rows = df[df["invalid_interval"] == True].copy()

    print(f"{name}: Invalid intervals = {len(invalid_rows)}")

    df_clean = df[df["invalid_interval"] == False].copy()

    return df_clean, invalid_rows


# 4. Detect overlaps per hole
def detect_overlaps(df, name):
    df = df.sort_values(["hole_id", "depth_from"])
    df["overlap"] = df.groupby("hole_id")["depth_from"].shift(-1) < df["depth_to"]

    overlap_rows = df[df["overlap"] == True].copy()
    print(f"{name}: Overlapping intervals = {len(overlap_rows)}")

    df_clean = df.copy()  # you may remove overlaps optionally

    return df_clean, overlap_rows


# 5. Validate chemical thresholds
def validate_chem(df, thresholds, name):
    issue_frames = []

    for col, (low, high) in thresholds.items():
        if col in df.columns:
            df[f"{col}_unrealistic"] = (df[col] < low) | (df[col] > high)
            invalid_rows = df[df[f"{col}_unrealistic"] == True].copy()

            print(f"{name}: {col} unrealistic = {len(invalid_rows)}")

            if len(invalid_rows) > 0:
                invalid_rows["issue_column"] = col
                issue_frames.append(invalid_rows)

    # Combine all unrealistic data rows
    if len(issue_frames) > 0:
        combined_issues = pd.concat(issue_frames, ignore_index=True)
    else:
        combined_issues = pd.DataFrame()

    return df, combined_issues


In [ ]:
import pandas as pd
import numpy as np

# ------------------------------------------------------------
# Helper functions
# ------------------------------------------------------------

def divider(title):
    print(f"\n{'='*20} {title} {'='*20}\n")

# 1. Duplicate removal
def remove_duplicates(df, subset):
    before = len(df)

    # Identify duplicates BEFORE removal
    duplicates = df[df.duplicated(subset=subset, keep=False)].copy()

    df_clean = df.drop_duplicates(subset=subset)
    removed = before - len(df_clean)

    print(f"Removed {removed} duplicates based on {subset}")

    return df_clean, duplicates


# 2. Normalize missing values
def normalize_missing(df, essential_cols=None):
    error_tokens = ["#N/A", "N/A", "NA", "", "-", "--", "null", "NULL"]
    df = df.replace(error_tokens, np.nan)

    if essential_cols:
        missing_rows = df[df[essential_cols].isna().any(axis=1)].copy()
        df_clean = df.dropna(subset=essential_cols)
    else:
        missing_rows = pd.DataFrame()
        df_clean = df

    print(f"Missing essential rows removed: {len(missing_rows)}")

    return df_clean, missing_rows


# 3. Validate intervals
def validate_intervals(df, name):
    if not {"depth_from", "depth_to"}.issubset(df.columns):
        print(f"{name}: missing depth interval columns.")
        return df, pd.DataFrame()

    df["invalid_interval"] = (df["depth_from"] >= df["depth_to"])

    invalid_rows = df[df["invalid_interval"] == True].copy()

    print(f"{name}: Invalid intervals = {len(invalid_rows)}")

    df_clean = df[df["invalid_interval"] == False].copy()

    return df_clean, invalid_rows


# 4. Detect overlaps per hole
def detect_overlaps(df, name):
    df = df.sort_values(["hole_id", "depth_from"])
    df["overlap"] = df.groupby("hole_id")["depth_from"].shift(-1) < df["depth_to"]

    overlap_rows = df[df["overlap"] == True].copy()
    print(f"{name}: Overlapping intervals = {len(overlap_rows)}")

    df_clean = df.copy()  # data are retained; user may correct overlaps manually

    return df_clean, overlap_rows


# 5. Validate chemical thresholds
def validate_chem(df, thresholds, name):
    issue_frames = []

    for col, (low, high) in thresholds.items():
        if col in df.columns:
            df[f"{col}_unrealistic"] = (df[col] < low) | (df[col] > high)
            invalid_rows = df[df[f"{col}_unrealistic"] == True].copy()

            print(f"{name}: {col} unrealistic = {len(invalid_rows)}")

            if len(invalid_rows) > 0:
                invalid_rows["issue_column"] = col
                issue_frames.append(invalid_rows)

    # Combine all unrealistic rows
    if len(issue_frames) > 0:
        combined_issues = pd.concat(issue_frames, ignore_index=True)
    else:
        combined_issues = pd.DataFrame()

    return df, combined_issues
    
    
 # ============================================================
# CLEANING EXECUTION + COLLECT ISSUE ROWS (WITH TIMESTAMP EXPORT)
# ============================================================

import os
from datetime import datetime

# Create folder for QAQC issue logs
os.makedirs("log_error", exist_ok=True)

# Timestamp format for filenames
ts = datetime.now().strftime("%Y%m%d_%H%M")

all_issues = {}   # dictionary to collect all problematic rows

# ---------------- Duplicate Removal ----------------
collar_df, collar_dups = remove_duplicates(collar_df, ["hole_id"])
assay_df,  assay_dups  = remove_duplicates(assay_df, ["sample_id"])
geo_df,    geo_dups    = remove_duplicates(geo_df, ["sample_id"])

all_issues["collar_duplicates"] = collar_dups
all_issues["assay_duplicates"]  = assay_dups
all_issues["geo_duplicates"]    = geo_dups

# ---------------- Missing Values -------------------
collar_df, collar_missing = normalize_missing(collar_df, ["hole_id"])
assay_df,  assay_missing  = normalize_missing(assay_df, ["sample_id", "hole_id"])
geo_df,    geo_missing    = normalize_missing(geo_df,   ["sample_id", "hole_id"])

all_issues["collar_missing"] = collar_missing
all_issues["assay_missing"]  = assay_missing
all_issues["geo_missing"]    = geo_missing

# ---------------- Interval Validation --------------
assay_df, invalid_assay_int = validate_intervals(assay_df, "Assay")
geo_df,   invalid_geo_int   = validate_intervals(geo_df,   "Lithology")

all_issues["assay_invalid_interval"] = invalid_assay_int
all_issues["geo_invalid_interval"]   = invalid_geo_int

# ---------------- Overlap Detection ----------------
assay_df, overlap_assay = detect_overlaps(assay_df, "Assay")
geo_df,   overlap_geo   = detect_overlaps(geo_df,   "Lithology")

all_issues["assay_overlap"] = overlap_assay
all_issues["geo_overlap"]   = overlap_geo

# ---------------- Chemical Validation --------------
thresholds = {
    "ni":  (0, 5),
    "fe":  (0, 60),
    "mgo": (0, 40),
    "sio2": (0, 60),
}

assay_df, chem_issues = validate_chem(assay_df, thresholds, "Assay")
all_issues["chemical_unrealistic"] = chem_issues

# ============================================================
# EXPORT ALL ISSUE LOGS WITH TIMESTAMP
# ============================================================
print("\n=== EXPORTING ISSUE LOGS ===")

for key, df_issue in all_issues.items():
    if df_issue is not None and len(df_issue) > 0:
        filename = f"log_error/{key}_{ts}.csv"
        df_issue.to_csv(filename, index=False)
        print(f"Saved issue: {filename}  ({len(df_issue)} rows)")
    else:
        print(f"No issues for: {key}")


Removed 0 duplicates based on ['hole_id']
Removed 0 duplicates based on ['sample_id']
Removed 0 duplicates based on ['sample_id']
Missing essential rows removed: 0
Missing essential rows removed: 0
Missing essential rows removed: 0
Assay: Invalid intervals = 0
Lithology: Invalid intervals = 0
Assay: Overlapping intervals = 0
Lithology: Overlapping intervals = 0
Assay: ni unrealistic = 0
Assay: fe unrealistic = 0
Assay: mgo unrealistic = 200
Assay: sio2 unrealistic = 20

=== EXPORTING ISSUE LOGS ===
No issues for: collar_duplicates
No issues for: assay_duplicates
No issues for: geo_duplicates
No issues for: collar_missing
No issues for: assay_missing
No issues for: geo_missing
No issues for: assay_invalid_interval
No issues for: geo_invalid_interval
No issues for: assay_overlap
No issues for: geo_overlap
Saved issue: QAQC_Reports/chemical_unrealistic_20251120_2047.csv  (220 rows)
